In [0]:
%pip install databricks-labs-dqx

In [0]:
dbutils.library.restartPython()

In [0]:
raw_posts_df = spark.read.table("default.raw_posts")

In [0]:

from databricks.labs.dqx import check_funcs
from databricks.labs.dqx.engine import DQEngine
from databricks.labs.dqx.rule import DQForEachColRule, DQRowRule
from databricks.sdk import WorkspaceClient

In [0]:
dq_engine = DQEngine(WorkspaceClient())

checks = [
    *DQForEachColRule(
        columns=["Id", "CreationDate"],
        criticality="error",
        check_func=check_funcs.is_not_null
    ).get_rules(),
    DQRowRule(
        name="creation_date_not_in_future",
        criticality="error",
        check_func=check_funcs.is_not_in_future,
        column="CreationDate"
    ),
    DQRowRule(
        name="post_type_id_allowed_values",
        criticality="warn",
        check_func=check_funcs.is_in_list,
        column="PostTypeId",
        check_func_kwargs={"allowed": ["1", "2", "3", "4"]}
    )
]

In [0]:
valid_df, quarantined_df = dq_engine.apply_checks_and_split(raw_posts_df, checks)
     